# Ejercicio adicional de fin de semana: semana 2

Ahora usa todo lo que aprendiste en la semana 2 para construir un prototipo completo para la pregunta/respuesta técnica que creaste en el ejercicio de la semana 1.

Esto debería incluir una interfaz de usuario de Gradio, transmisión, uso del mensaje del sistema para agregar experiencia y la capacidad de cambiar entre modelos. ¡Puntos extra si puedes demostrar el uso de una herramienta!

Si te sientes audaz, ve si puedes agregar una entrada de audio para poder hablarle y hacer que responda con audio. ChatGPT o Claude pueden ayudarte, o envíame un correo electrónico si tienes preguntas.

Pronto publicaré una solución completa aquí, a menos que alguien se me adelante...

Hay tantas aplicaciones comerciales para esto, desde un tutor de idiomas hasta una solución de incorporación de empresas, pasando por una IA complementaria para un curso (¡como este!). No puedo esperar a ver tus resultados.

In [1]:
# Asistente Técnico de Python con IA
# Incluye: Gradio UI, Streaming, Sistema de mensajes, Cambio de modelos, Herramientas y Audio

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from io import BytesIO
from pydub import AudioSegment
import base64
import tempfile
import subprocess
import time

In [2]:
# Inicialización
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key configurada correctamente")
else:
    print("OpenAI API Key sin configurar")

openai = OpenAI()
MODEL = "gpt-4o-mini"
# Sistema de mensajes especializado
system_message = """Eres un asistente técnico experto en Python. 
Tus especialidades son:
- Explicar conceptos de programación de forma clara
- Ayudar a debuggear código
- Sugerir mejores prácticas
- Proporcionar ejemplos prácticos

Da respuestas precisas y bien estructuradas. Si necesitas buscar documentación específica, 
usa las herramientas disponibles. Siempre proporciona código limpio y comentado."""

OpenAI API Key configurada correctamente


In [3]:
# Herramientas disponibles

# Base de datos simulada de documentación
python_docs = {
    "listas": "Las listas en Python son colecciones ordenadas y mutables. Métodos principales: append(), extend(), insert(), remove(), pop(), clear(), index(), count(), sort(), reverse()",
    "diccionarios": "Los diccionarios son colecciones de pares clave-valor. Métodos principales: keys(), values(), items(), get(), pop(), update(), clear()",
    "strings": "Strings son secuencias inmutables de caracteres. Métodos principales: upper(), lower(), strip(), split(), join(), replace(), find(), format()",
    "funciones": "Las funciones se definen con 'def'. Pueden tener parámetros, valores por defecto, *args y **kwargs. Usar docstrings para documentar.",
    "clases": "Las clases definen objetos con __init__ como constructor. Soportan herencia, métodos especiales (__str__, __repr__) y propiedades.",
    "excepciones": "Manejo de errores con try/except/finally. Crear excepciones personalizadas heredando de Exception. Usar 'raise' para lanzar errores."
}

def get_python_documentation(topic):
    """Busca documentación sobre un tema específico de Python"""
    print(f"Herramienta llamada: get_python_documentation('{topic}')")
    topic_lower = topic.lower()
    
    for key in python_docs:
        if key in topic_lower:
            return python_docs[key]
    
    return f"No se encontró documentación específica para '{topic}'. Temas disponibles: {', '.join(python_docs.keys())}"

In [4]:
def analyze_code(code):
    """Analiza código Python y da feedback básico"""
    print(f"Herramienta llamada: analyze_code")
    
    feedback = []
    lines = code.split('\n')
    
    # Análisis básico
    if len(lines) > 100:
        feedback.append("El código es extenso (>100 líneas). Considera dividirlo en funciones.")
    
    if 'def ' in code:
        feedback.append("Buena práctica: Uso de funciones")
    
    if '# ' in code or '"""' in code:
        feedback.append("Buena práctica: Código documentado")
    
    if 'try:' in code and 'except' in code:
        feedback.append("Buena práctica: Manejo de excepciones")
    
    if not feedback:
        feedback.append("Código básico detectado. Considera añadir documentación y manejo de errores.")
    
    return " | ".join(feedback)

# Definición de herramientas para OpenAI
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_python_documentation",
            "description": "Busca documentación oficial sobre un tema específico de Python (listas, diccionarios, strings, funciones, clases, excepciones)",
            "parameters": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "El tema de Python sobre el que buscar documentación"
                    }
                },
                "required": ["topic"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_code",
            "description": "Analiza código Python y proporciona feedback sobre buenas prácticas",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "El código Python a analizar"
                    }
                },
                "required": ["code"],
                "additionalProperties": False
            }
        }
    }
]

In [ ]:
# Manejo de llamadas a herramientas
def handle_tool_call(message):
    """Procesa las llamadas a herramientas del modelo"""
    # print("tools: ", message.tool_calls[0])
    tool_call = message.tool_calls[0]

    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    
    # Ejecutar la función correspondiente
    if function_name == "get_python_documentation":
        result = get_python_documentation(arguments.get('topic'))
    elif function_name == "analyze_code":
        result = analyze_code(arguments.get('code'))
    else:
        result = "Función no reconocida"
    
    response = {
        "role": "tool",
        "content": json.dumps({"result": result}),
        "tool_call_id": tool_call.id
    }
    
    return response

In [6]:
def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(3) # El estudiante Dominic consideró que esto era necesario. También podrías intentar comentar para ver si no es necesario en tu PC
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass
 
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Prueba también de reemplazar onyx por alloy
        input=message
    )
    audio_stream = BytesIO(response.content)# El audio está codificado en bytes por tanto hay que decodificar a un formato de audio especifico
    audio = AudioSegment.from_file(audio_stream, format="mp3") # decodificamos a mp3
    play_audio(audio)

In [11]:
def chat(history, enable_audio):
    """Función principal de chat con opción de audio"""
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Reproducir audio solo si está habilitado
    if enable_audio:
        talker(reply)
    
    return history

In [13]:
with gr.Blocks() as ui:
    gr.Markdown("# 🐍 Asistente Técnico de Python con IA")
    gr.Markdown("Pregunta sobre Python, envía código para analizar, o solicita documentación específica.")
    
    with gr.Row():
        chatbot = gr.Chatbot(height=500, label="Conversación")
    
    with gr.Row():
        entry = gr.Textbox(
            label="Escribe tu mensaje aquí:", 
            placeholder="Ej: ¿Cómo funcionan las listas en Python?",
            lines=2,
            scale=4
        )
        send_btn = gr.Button("📤 Enviar", scale=1)
    
    with gr.Row():
        # Checkbox para habilitar/deshabilitar audio
        audio_toggle = gr.Checkbox(
            label="🔊 Habilitar respuestas de voz", 
            value=False,
            info="Activa para escuchar las respuestas del asistente"
        )
        clear = gr.Button("🗑️ Limpiar Chat")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(
        do_entry, 
        inputs=[entry, chatbot], 
        outputs=[entry, chatbot]
    ).then(
        chat, 
        inputs=[chatbot, audio_toggle],  # Ahora pasamos el estado del checkbox
        outputs=[chatbot]
    )
    
    send_btn.click(
        do_entry,
        inputs=[entry, chatbot],
        outputs=[entry, chatbot]
    ).then(
        chat,
        inputs=[chatbot, audio_toggle],
        outputs=[chatbot]
    )
    
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Herramienta llamada: get_python_documentation('listas')
Herramienta llamada: analyze_code
Herramienta llamada: analyze_code
